In [ ]:
import os
import sys
from datetime import datetime

# Add the path to the geoscience-objects helpers directory
sys.path.append("./helpers")
import evo_common

from evo.blockmodels import BlockModelAPIClient
from evo.notebooks import ServiceManagerWidget

cache_location = "./notebook-data"
download_path = cache_location + "/downloads"
input_path = f"{cache_location}/input"

if not os.path.exists(download_path):
    try:
        os.mkdir(download_path)
    except Exception:
        raise RuntimeError(f"Error: Failed to create {download_path}.")

# Evo app credentials
client_id = "daves-evo-client"
redirect_url = "http://localhost:32369/auth/callback"

manager = await ServiceManagerWidget.with_auth_code(
    discovery_url="https://discover.api.seequent.com",
    redirect_url=redirect_url,
    client_id=client_id,
    cache_location=cache_location,
).login()

## Prepare Evo SDK parameters

In [ ]:
# Get the environment and connector from the ServiceManagerWidget instance.
# The environment contains the hub URL, organization ID, and workspace ID.
# The connector is used to make API calls to the Evo service.
environment = manager.get_environment()
connector = manager.get_connector()

service_client = BlockModelAPIClient(environment, connector, manager.cache)

# Create a BlockModelSelector instance. This will be used to select block models from the service.
# The source code for this class is in helpers/evo_common.py
bm_selector = evo_common.BlockModelSelector(environment, connector)

workspace_id = None
org_id = None

## Demo 1: Create a regular block model

In [ ]:
import pyarrow

from evo.blockmodels.data import RegularGridDefinition
from evo.blockmodels.endpoints.models import RotationAxis

name = f"Regular block model {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"
description = "This is a description of my regular block model."
object_path = "/Jupyter/SDK-demo/"  # Similar to a folder path.
coordinate_reference_system = "EPSG:3395"  # BlockSync only supports EPSG codes that are meter based.
size_unit_id = "m"

grid_definition = RegularGridDefinition(
    model_origin=[1478500, 5174500, 100],
    rotations=[(RotationAxis.x, 0), (RotationAxis.y, 0), (RotationAxis.z, 0)],  # Null rotation for demo purposes
    n_blocks=[48, 68, 40],
    block_size=[25, 25, 25],
)

initial_data = pyarrow.table(
    {
        "x": [1478512.5, 1479687.5, 1479612.5],
        "y": [5174512.5, 5175087.5, 5175137.5],
        "z": [112.5, 337.5, 587.5],
        "Au": ["A", "D", "E"],
        "Cu": [1.5, 1.3, 1.2],
    },
    schema=pyarrow.schema(
        {
            "x": pyarrow.float64(),
            "y": pyarrow.float64(),
            "z": pyarrow.float64(),
            "Au": pyarrow.string(),
            "Cu": pyarrow.float64(),
        }
    ),
)

block_model, version = await service_client.create_block_model(
    name=name,
    description=description,
    grid_definition=grid_definition,
    object_path=object_path,
    coordinate_reference_system=coordinate_reference_system,
    size_unit_id=size_unit_id,
    initial_data=initial_data,
)

print(f"Created block model with ID: {block_model.id}")

## Demo 2: Add new columns to a block model

In [ ]:
new_cols = pyarrow.table(
    {
        "x": [1478512.5, 1479687.5, 1479612.5],
        "y": [5174512.5, 5175087.5, 5175137.5],
        "z": [112.5, 337.5, 587.5],
        "Ag": [120, 155, 72],
        "Pt": [4.5, 5.3, 2.1],
    },
    schema=pyarrow.schema(
        {
            "x": pyarrow.float64(),
            "y": pyarrow.float64(),
            "z": pyarrow.float64(),
            "Ag": pyarrow.int32(),
            "Pt": pyarrow.float64(),
        }
    ),
)

# Add new columns to the block model and include units for the "Pt" column.
version_with_new_columns = await service_client.add_new_columns(
    bm_id=block_model.id,
    data=new_cols,
    units={"Pt": "g/t"},
)

print(f"Updated block model with new columns: {[col for col in new_cols.column_names if col not in ['x', 'y', 'z']]}")